# Generation of professional movements with ATT-RGOM

In this Jupyter Notebook, we will see the modeling of human movements using an Autoencoder with Luong attention (global).
The AE computes the time-varying coefficients of the full-body GOM representations, which are then used to generate professional movements.

The script is divided into six sections:

**1.** Loading of the libraries and pickles files containing the joint angles of seven datasets and their labels 

**2.** Obtaining the indexes for the train set and test set using stratified cross-validation

**3.** The dataset merged from all seven datasets is preprocessed

**4.** Define the structure of the ATT-RGOM network

**5.** Training and validation of ATT-RGOM using the training and testing indexes off the 5-fold cross-validation

**6.** Results of the testing tests for each of the seven datasets

**Note that the full description of this deep state-space model is provided in** [(1)](https://arxiv.org/abs/2304.14502)
    

###  1. Load the libraries and motion data of the seven datasets with professional tasks:


In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import pickle 
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv1D, Layer, LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot, Lambda, Reshape, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
import math
from sklearn.metrics import mean_squared_error, pairwise_distances, mean_absolute_error
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from scipy.spatial.distance import cdist, euclidean

# The local joint angles of each dataset are saved in pickle files. 
# Each pickle file contains the data frames of the movement iterations, and their paths are indicated in the file_list.

with open('ERGD_EulerLocalAngles.pkl', 'rb') as f: [data_ERGD, file_list_ERGD, usedJoints] = pickle.load(f)
with open('APA_EulerLocalAngles.pkl', 'rb') as f: [data_APA, file_list_APA, _] = pickle.load(f)
with open('TVA_EulerLocalAngles.pkl', 'rb') as f: [data_TVA, file_list_TVA, _] = pickle.load(f)
with open('TVP_EulerLocalAngles.pkl', 'rb') as f: [data_TVP, file_list_TVP, _] = pickle.load(f)
with open('SLW_EulerLocalAngles.pkl', 'rb') as f: [data_SLW, file_list_SLW, _] = pickle.load(f)
with open('GLB_EulerLocalAngles.pkl', 'rb') as f: [data_GLB, file_list_GLB, _] = pickle.load(f)
with open('MSC_EulerLocalAngles.pkl', 'rb') as f: [data_MSC, file_list_MSC, _] = pickle.load(f)

angles_labels = data_ERGD[0].columns

# All datasets are merged into one, and their corresponding labels are indicated in the list "labels" from the pickle file.

data = data_ERGD+data_APA+data_TVA+data_TVP+data_SLW+data_GLB+data_MSC

file_list = np.concatenate((file_list_ERGD, file_list_APA,file_list_TVA,file_list_TVP,file_list_SLW,file_list_GLB,file_list_MSC))


with open('labels_allFiles_set.pkl', 'rb') as f: [labels, _] = pickle.load(f)

###  2. Obtain the indexes of the train set and test set. 

As there are more iterations in some professional tasks than others, stratified cross-validation is applied to evaluate the model's performance.

**Note** `random_state=1` in order to obtain the same indexes and for future comparing the performance of other models in the generation of the professional movements.

In [2]:
# 5-FOLD 
train_set = []
test_set = []
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state=1)

for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
    train_set.append(train_index)
    test_set.append(test_index)

C:\Users\aimove\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


### 3. Preprocessing of the data for ATT-RGOM

In this cell, the data is normalized and divided by overlapping windows of three seconds. Then, these windows are processed by the network for making one-step predictions.


In [3]:
scalers = {} # The scalers for each joint angle are saved for the future inverse transformation
dataX =[]  # The overlapping three-second windows are placed in dataX
dataY =[] # The prediction of the corresponding windows is placed in dataY

for dd in range(0,len(data)):
    dat = data[dd].values[1:,:]  
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(dat)
    scalers[dd] = scaler
    ned = scaler.transform(dat)
        
    wx = []
    wy = []
    for w in range(0,len(dat)-3, 1):
        wx.append(np.arange(0+w,3+w))
        wy.append(np.arange(3+w,4+w))
    
    dX=[]
    dY = []
    for wi in range(0,len(wx)):
        dX.append(ned[wx[wi],:])
        dY.append(ned[wy[wi],:])
    dataX.append(np.array(dX))
    dataY.append(np.array(dY))

### 4. ATT-RGOM

**4.1** Next is defined the architecture of the encoder and decoder that compose ATT-RGOM, as well as the functions used for the one-step prediction. The hyperparameters were previously selected using a Bayesian optimization.

In [4]:
# Hadmard product is applied to solve the GOM representations given the tensor of coefficients provided by the decode
def hadamard_product(x):
    coef = x[0]
    inp = x[1]
    inp2 = K.expand_dims(inp, axis=1)
    m1 = coef*inp2
    m2 = K.sum(m1, axis=(2,3)) 
    y = K.expand_dims(m2, axis=-2)
    return y

n_hidden = 32 
hp_act2 = 'softsign' 
hp_act1 = 'softsign' 
w=3
    
input_train = Input(shape=(dataX[0].shape[1], dataX[0].shape[2]))
output_train = Input(shape=(dataY[0].shape[1], dataY[0].shape[2]))

# ENCODER
encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(n_hidden, activation= hp_act1, dropout=0.2, recurrent_dropout=0.2, 
return_state=True, return_sequences=True)(input_train)
encoder_last_h = BatchNormalization()(encoder_last_h)
encoder_last_c = BatchNormalization()(encoder_last_c)

# DECODER
decoder_input = RepeatVector(input_train.shape[1])(encoder_last_h)
decoder_stack_h = LSTM(n_hidden, activation=hp_act1, dropout=0.2, recurrent_dropout=0.2,
 return_state=False, return_sequences=True)(decoder_input, initial_state=[encoder_last_h, 
                                                                  encoder_last_c])
# ATTENTION
attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
attention = Activation('softmax')(attention)

context = dot([attention, encoder_stack_h], axes=[2,1])
decoder_combined_context = concatenate([context, decoder_stack_h])


xC = TimeDistributed(Dense(dataY[0].shape[2]*dataY[0].shape[2]))(decoder_combined_context)
xC_input = Reshape((dataY[0].shape[2], w, dataY[0].shape[2]))(xC)

# Solve GOM
out = Lambda(hadamard_product, output_shape=(dataY[0].shape[2],))([xC_input, input_train])


model = Model(inputs=input_train, outputs=[out])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 57)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 3, 32), (Non 11520       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32)           128         lstm[0][1]                       
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 3, 32)        0           batch_normalization[0][0]        
______________________________________________________________________________________________

**4.2** On the following cell are defined the metrics used for the training. A prediction loss is only used, which consists of the Mean Squared Error. The Mean Absolute Error is calculated for observation during training:

In [5]:
def euler_angles_loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError(reduction=keras.losses.Reduction.SUM)
    
    return tf.reduce_mean(mse(y_true, y_pred))

def mae_loss(y_true, y_pred):
    mae = tf.keras.losses.MeanAbsoluteError(reduction=keras.losses.Reduction.SUM)
    
    return tf.reduce_mean(mae(y_true, y_pred))

### 5. Evaluation  of the model using a stratified 5-fold cross-validation
The train and test indexes define each iteration's train set and test set. Then, save the results from analyzing the performance achieved with each professional movement.

In [7]:
kResults = []
for k in range(0, len(train_set)):
    print(f"Fold {k}:")
    train_index = train_set[k] # Obtain the train indexes for the iteration k
    test_index = test_set[k] # Obtain the test indexes for the iteration k
    X_train_cv = [dataX[tt] for tt in train_index]
    y_train_cv = [dataY[tt] for tt in train_index]
    
    X_test_cv = [dataX[tt] for tt in test_index]
    y_test_cv= [dataY[tt] for tt in test_index]
    scaler_Test = [scalers[tt] for tt in test_index]
    file_list_Test = [file_list[tt] for tt in test_index]
    lables_Test = [labels[tt] for tt in test_index]
    
    X_input_train = np.concatenate(X_train_cv, axis=0)
    X_output_train = np.concatenate(y_train_cv, axis=0)
    
    X_input_train, X_output_train = shuffle(X_input_train, X_output_train)
    
    model = Model(inputs=input_train, outputs=[out]) # Create ATT-RGOM
    opt = Adam(learning_rate=0.0001, clipnorm=1)
    model.compile(loss=euler_angles_loss, optimizer=opt, metrics=[euler_angles_loss, mae_loss])
    
    epc = 50
    es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
    history = model.fit(X_input_train, X_output_train, validation_split=0.1, 
                        epochs=epc, verbose=1, callbacks=[es], 
                        batch_size=1024) # Training of the model
    
    # Evaluate the performance of the trained model with the test set
    metrics = pd.DataFrame() # Save results in the metrics data frame
    for f in range(0, len(X_test_cv)):
        df_test = X_test_cv[f]
        y = y_test_cv[f]
        scaler_T = scaler_Test[f]
        
        y=y.reshape([y.shape[0],y.shape[2]])
        
        
        d1_pred = model.predict(df_test)
        pred_ATT = keras.backend.get_value(d1_pred)
        
        pred_ATT=pred_ATT.reshape([pred_ATT.shape[0],pred_ATT.shape[2]])
        
        pred_ATT = scaler_T.inverse_transform(pred_ATT)
        y = scaler_T.inverse_transform(y)
        
        # Metrics used for the evaluation: 
        mse_t = mean_squared_error(y, pred_ATT) # Mean Squared Error
        mae_t = mean_absolute_error(y, pred_ATT) # Mean Absolute Error
        pde_t = [cdist(y[:,ii].reshape(1,-1), pred_ATT[:,ii].reshape(1,-1), 'euclidean') for ii in range(0,y.shape[1])] # Average Pairwise Distance
        fde_t = [euclidean(y[-1,ii], pred_ATT[-1,ii]) for ii in range(0,y.shape[1])] # Final Displacement Error
        
        met_f = pd.Series({
                'File' : file_list_Test[f],
                'Class' : lables_Test[f],
                'MSE' : mse_t,
                'MAE' : mae_t,
                'APD' : np.mean(pde_t),
                'AFDE': np.mean(fde_t),
            })
        
        metrics = pd.concat([metrics, met_f], axis = 1)
        
    kResults.append(metrics)

Fold 0:
Epoch 1/50
1125/1125 [==============================] - 41s 32ms/step - loss: 0.4069 - euler_angles_loss: 0.4069 - mae_loss: 8.4988 - val_loss: 0.3468 - val_euler_angles_loss: 0.3468 - val_mae_loss: 6.6503
Epoch 2/50
1125/1125 [==============================] - 35s 31ms/step - loss: 0.4026 - euler_angles_loss: 0.4026 - mae_loss: 8.3627 - val_loss: 0.3538 - val_euler_angles_loss: 0.3538 - val_mae_loss: 7.0468
Epoch 3/50
1125/1125 [==============================] - 35s 32ms/step - loss: 0.4001 - euler_angles_loss: 0.4001 - mae_loss: 8.2995 - val_loss: 0.3391 - val_euler_angles_loss: 0.3391 - val_mae_loss: 6.1395
Epoch 4/50
1125/1125 [==============================] - 35s 31ms/step - loss: 0.3983 - euler_angles_loss: 0.3983 - mae_loss: 8.2446 - val_loss: 0.3439 - val_euler_angles_loss: 0.3439 - val_mae_loss: 6.5065
Epoch 5/50
1125/1125 [==============================] - 35s 32ms/step - loss: 0.3967 - euler_angles_loss: 0.3967 - mae_loss: 8.1814 - val_loss: 0.3427 - val_euler_angle

Epoch 34/50
1141/1141 [==============================] - 36s 32ms/step - loss: 0.3607 - euler_angles_loss: 0.3605 - mae_loss: 5.6152 - val_loss: 0.3707 - val_euler_angles_loss: 0.3704 - val_mae_loss: 5.5182
Epoch 35/50
1141/1141 [==============================] - 37s 33ms/step - loss: 0.3603 - euler_angles_loss: 0.3601 - mae_loss: 5.6046 - val_loss: 0.3677 - val_euler_angles_loss: 0.3673 - val_mae_loss: 5.3374
Epoch 36/50
1141/1141 [==============================] - 36s 32ms/step - loss: 0.3598 - euler_angles_loss: 0.3596 - mae_loss: 5.5962 - val_loss: 0.3693 - val_euler_angles_loss: 0.3690 - val_mae_loss: 5.4738
Epoch 37/50
1141/1141 [==============================] - 36s 32ms/step - loss: 0.3593 - euler_angles_loss: 0.3591 - mae_loss: 5.5873 - val_loss: 0.3679 - val_euler_angles_loss: 0.3676 - val_mae_loss: 5.4143
Epoch 38/50
1141/1141 [==============================] - 37s 32ms/step - loss: 0.3589 - euler_angles_loss: 0.3587 - mae_loss: 5.5757 - val_loss: 0.3697 - val_euler_angles_l

In [8]:
# SAVE RESULTS AND MODEL
#with open('ATTRGOM_kfold_results.pkl', 'wb') as f: pickle.dump([kResults], f)
#model.save('ATTRGOM_model')

#LOAD RESULTS
#with open('ATTRGOM_kfold_results.pkl', 'rb') as f: [kResults] = pickle.load(f)

INFO:tensorflow:Assets written to: ATTRGOM_model\assets


### 6. Metrics obtained for the movements of each dataset:

In [9]:
# ERGD
c=28
classD = np.arange(0,c) # Labels of the movements from the ERGD dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
ERGD_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
ERGD_results.describe()

,Class,MSE,MAE,APD,AFDE
count,28.000000,27.000000,27.000000,27.000000,27.000000
mean,13.500000,3.875419,0.061090,7.776831,0.074605
std,8.225975,3.578266,0.036457,6.023520,0.100889
min,0.000000,0.000908,0.003286,0.165758,0.003180
25%,6.750000,1.087875,0.028925,2.507833,0.027853
50%,13.500000,2.913476,0.061388,7.617158,0.051041
75%,20.250000,6.095881,0.086025,11.085939,0.071550
max,27.000000,14.592700,0.143800,23.477655,0.514780


In [10]:
# APA
c = c+3
classD = np.arange(c-3,c) # Labels of the movements from the APA dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
APA_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
APA_results.describe()

,Class,MSE,MAE,APD,AFDE
count,3.0,3.000000,3.000000,3.000000,3.000000
mean,29.0,1.765285,0.085872,16.431264,0.253271
std,1.0,0.630725,0.019621,10.076877,0.249022
min,28.0,1.329538,0.064010,6.914815,0.091643
25%,28.5,1.403659,0.077832,11.152956,0.109885
50%,29.0,1.477780,0.091654,15.391098,0.128127
75%,29.5,1.983159,0.096803,21.189489,0.334085
max,30.0,2.488538,0.101952,26.987881,0.540044


In [11]:
# TVA
c = c+4
classD = np.arange(c-4,c) # Labels of the movements from the TVA dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
TVA_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
TVA_results.describe()

,Class,MSE,MAE,APD,AFDE
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,32.500000,3.111336,0.080427,5.726399,0.108359
std,1.290994,2.039208,0.015272,3.449881,0.028243
min,31.000000,0.449022,0.057604,3.642511,0.086101
25%,31.750000,2.407621,0.079168,3.713206,0.095522
50%,32.500000,3.302687,0.087279,4.204525,0.098786
75%,33.250000,4.006402,0.088538,6.217717,0.111623
max,34.000000,5.390950,0.089548,10.854036,0.149763


In [12]:
# TVP
c = c+9
classD = np.arange(c-9,c) # Labels of the movements from the TVP dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
TVP_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
TVP_results.describe()

,Class,MSE,MAE,APD,AFDE
count,9.000000,9.000000,9.000000,9.000000,9.000000
mean,39.000000,13.176786,0.188429,53.490655,0.128084
std,2.738613,10.499009,0.064854,18.786016,0.025194
min,35.000000,1.960135,0.113395,27.964818,0.094163
25%,37.000000,5.314859,0.133849,38.155582,0.111628
50%,39.000000,9.840982,0.183093,54.455687,0.123933
75%,41.000000,16.226801,0.203550,72.237457,0.141823
max,43.000000,30.768323,0.293538,78.408731,0.170307


In [13]:
# SLW
c = c+16
classD = np.arange(c-16,c) # Labels of the movements from the SLW dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
SLW_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
SLW_results.describe()

,Class,MSE,MAE,APD,AFDE
count,16.000000,15.000000,15.000000,15.000000,15.000000
mean,51.500000,0.657582,0.076663,5.935919,0.086666
std,4.760952,1.373394,0.022194,8.725256,0.028441
min,44.000000,0.023453,0.047062,0.969070,0.054180
25%,47.750000,0.039029,0.063250,1.506873,0.067009
50%,51.500000,0.069382,0.070663,2.224926,0.075509
75%,55.250000,0.388154,0.082462,5.027869,0.105048
max,59.000000,4.991017,0.134197,30.823876,0.145745


In [14]:
# GLB
c = c+18
classD = np.arange(c-18,c) # Labels of the movements from the GLB dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
GLB_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
GLB_results.describe()

,Class,MSE,MAE,APD,AFDE
count,18.000000,18.000000,18.000000,18.000000,18.000000
mean,68.500000,7.659004,0.107300,13.731455,0.242279
std,5.338539,7.849738,0.046044,8.838128,0.263192
min,60.000000,0.017424,0.055775,1.333161,0.082671
25%,64.250000,3.484244,0.080082,8.873583,0.121969
50%,68.500000,4.602765,0.090869,10.651502,0.153152
75%,72.750000,9.784604,0.119748,18.082966,0.211201
max,77.000000,31.783086,0.245424,34.253981,1.144325


In [15]:
# MSC
c = c+13
classD = np.arange(c-13,c) # Labels of the movements from the MSC dataset
dflist =[]
mean_MSE=[]; mean_MAE=[]; mean_APD=[]; mean_AFDE=[];
for i in classD:
    df = pd.DataFrame()
    for r in kResults:
        r2 = r.T.reset_index(drop=True)
        df1 = r2[r2['Class'] == i]
        df = pd.concat([df, df1])
    dflist.append(df)
    mean_MSE.append(df['MSE'].mean())
    mean_MAE.append(df['MAE'].mean())
    mean_APD.append(df['APD'].mean())
    mean_AFDE.append(df['AFDE'].mean())
    
MSC_results = pd.DataFrame(data={'Class': classD, 'MSE':mean_MSE, 'MAE':mean_MAE, 'APD':mean_APD, 'AFDE':mean_AFDE})
MSC_results.describe()

,Class,MSE,MAE,APD,AFDE
count,13.00000,13.000000,13.000000,13.000000,13.000000
mean,84.00000,18.279810,0.203092,19.207933,0.266016
std,3.89444,19.007575,0.096628,14.090396,0.328711
min,78.00000,0.059326,0.098243,3.233970,0.072369
25%,81.00000,3.541428,0.139354,10.482957,0.097360
50%,84.00000,12.263020,0.169324,14.313204,0.111568
75%,87.00000,28.345842,0.221940,23.210729,0.176718
max,90.00000,56.461435,0.393547,54.173298,1.035859
